In [ ]:
# Libraries to install
%pip install -U openai langgraph dotenv \
    langchain langchain-openai langchain-huggingface langchain-community \
    unstructured pymupdf python-docx \
    sentence_transformers numpy pandas networkx openpyxl faiss-cpu

  Using cached openai-1.97.1-py3-none-any.whl.metadata (29 kB)
  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_openai-0.3.28-py3-none-any.whl.metadata (2.3 kB)
  Using cached langchain_huggingface-0.3.1-py3-none-any.whl.metadata (996 bytes)
  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached unstructured-0.18.11-py3-none-any.whl.metadata (24 kB)
  Using cached pymupdf-1.26.3-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (3.4 kB)
  Using cached python_docx-1.2.0-py3-none-any.whl.metadata (2.0 kB)
  Using cached sentence_transformers-5.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached numpy-2.3.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached pandas-2.3.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached langchain_core-0.3.72-py3-none-any.whl.metadata